In [51]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)
    

In [52]:
import pyodbc
import pandas as pd

# connection 
conn = pyodbc.connect(
    "Driver={SQL Server};"
    "Server=DESKTOP-UR57B5H\\SQLEXPRESS;"  
    "Database=AdventureWorksLT2016;"
    "Trusted_Connection=yes;"
)

#test
df = pd.read_sql("SELECT TOP 10 * FROM dbo.buildversion;", conn)

C:\Users\HP\AppData\Local\Temp\ipykernel_21192\2410496830.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT TOP 10 * FROM dbo.buildversion;", conn)


In [53]:
saleslt_address=pd.read_sql('select*from saleslt.address',conn)

C:\Users\HP\AppData\Local\Temp\ipykernel_21192\2056209415.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  saleslt_address=pd.read_sql('select*from saleslt.address',conn)


In [54]:
saleslt_address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   AddressID      450 non-null    int64         
 1   AddressLine1   450 non-null    object        
 2   AddressLine2   11 non-null     object        
 3   City           450 non-null    object        
 4   StateProvince  450 non-null    object        
 5   CountryRegion  450 non-null    object        
 6   PostalCode     450 non-null    object        
 7   rowguid        450 non-null    object        
 8   ModifiedDate   450 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 31.8+ KB


In [55]:
saleslt_address.describe()

,AddressID,ModifiedDate
count,450.000000,450
mean,812.593333,2006-06-30 07:40:48
min,9.000000,2005-07-01 00:00:00
25%,550.250000,2005-09-01 00:00:00
50%,662.500000,2006-08-01 00:00:00
75%,995.750000,2007-02-01 00:00:00
max,11382.000000,2008-06-01 00:00:00
std,897.287273,NaN


In [56]:
saleslt_address.head()

,AddressID,AddressLine1,AddressLine2,City,StateProvince,CountryRegion,PostalCode,rowguid,ModifiedDate
0,9,8713 Yosemite Ct.,None,Bothell,Washington,United States,98011,268AF621-76D7-4C78-9441-144FD139821A,2006-07-01
1,11,1318 Lasalle Street,None,Bothell,Washington,United States,98011,981B3303-ACA2-49C7-9A96-FB670785B269,2007-04-01
2,25,9178 Jumping St.,None,Dallas,Texas,United States,75201,C8DF3BD9-48F0-4654-A8DD-14A67A84D3C6,2006-09-01
3,28,9228 Via Del Sol,None,Phoenix,Arizona,United States,85004,12AE5EE1-FC3E-468B-9B92-3B970B169774,2005-09-01
4,32,26910 Indela Road,None,Montreal,Quebec,Canada,H1Y 2H5,84A95F62-3AE8-4E7E-BBD5-5A6F00CD982D,2006-08-01


In [57]:
duplicates=saleslt_address[saleslt_address.duplicated(subset=['AddressID'],keep=False)]

print(len(duplicates)) 
# making sure there are no duplicates

0


In [58]:
saleslt_address['AddressID'].dtype

dtype('int64')

making sure it is a real primary key

In [59]:
saleslt_address["AddressLine1"] = saleslt_address["AddressLine1"].str.strip()
saleslt_address["AddressLine2"] = saleslt_address["AddressLine2"].str.strip()


In [60]:
#all nulls are replaced with unknown
saleslt_address['AddressLine2'].fillna('Unknown',inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_21192\631758378.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  saleslt_address['AddressLine2'].fillna('Unknown',inplace=True)


In [61]:

saleslt_address["City"].str.lower().value_counts()


City
toronto        24
london         14
montreal       13
calgary        11
mississauga     6
               ..
bountiful       1
daly city       1
riverton        1
tooele          1
cedar city      1
Name: count, Length: 269, dtype: int64

In [62]:

saleslt_address["City"].value_counts()

City
Toronto        24
London         14
Montreal       13
Calgary        11
Mississauga     6
               ..
Bountiful       1
Daly City       1
Riverton        1
Tooele          1
Cedar City      1
Name: count, Length: 269, dtype: int64

then all cities have standeralized capitalization


In [63]:
#no city is written in abbreviation form to make sure there are no duplicates
saleslt_address.groupby(saleslt_address["City"].str.len())["City"].unique()

City
4            [Hull, Oxon, York, Novi, Kent, Mesa, Reno]
5     [Edina, Elgin, Plano, Denby, Berks, Lacey, Sal...
6     [Dallas, Renton, Quebec, Weston, Ottawa, Surre...
7     [Bothell, Phoenix, Burnaby, Toronto, Calgary, ...
8     [Montreal, Bellevue, Richmond, Brossard, Winni...
9     [Vancouver, Etobicoke, Outremont, Wood Dale, S...
10    [Pnot-Rouge, Sainte-Foy, Saint John, North Yor...
11    [Chalk Riber, Scarborough, Mississauga, Saint ...
12    [West Chicago, Carol Stream, High Wycombe, Por...
13    [Richmond Hill, Johnson Creek, Milton Keynes, ...
14    [Ville De'anjou, Jefferson City, Corpus Christ...
15    [College Station, Gloucestershire, Madison Hei...
16                 [North Sioux City, City Of Commerce]
17                                  [Basingstoke Hants]
Name: City, dtype: object

In [64]:
# no stateprovince written in abbreviation form
saleslt_address.groupby(saleslt_address['StateProvince'].str.len())['StateProvince'].unique()

StateProvince
4                                                [Utah]
5                                        [Texas, Idaho]
6                              [Quebec, Oregon, Nevada]
7     [Arizona, Ontario, Alberta, England, Montana, ...
8     [Manitoba, Illinois, Missouri, Michigan, Color...
9                     [Brunswick, Minnesota, Wisconsin]
10                 [Washington, California, New Mexico]
12                                       [South Dakota]
16                                   [British Columbia]
Name: StateProvince, dtype: object

In [65]:
# converting to ISO code
saleslt_address['CountryRegion']=saleslt_address['CountryRegion'].replace('Canada','CAN')
saleslt_address['CountryRegion']=saleslt_address['CountryRegion'].replace('United States','USA')
saleslt_address['CountryRegion']=saleslt_address['CountryRegion'].replace('United Kingdom','UK')

In [66]:
# to make sure all in ISO code
saleslt_address.groupby(saleslt_address['CountryRegion'].str.len())['CountryRegion'].unique()

CountryRegion
2          [UK]
3    [USA, CAN]
Name: CountryRegion, dtype: object

In [67]:
#making sure all postal codes don't have unnecessary white spaces and all in capital letters
saleslt_address['PostalCode']=saleslt_address['PostalCode'].str.strip().str.upper()

In [68]:
# making sure all UUIDS are unique
duplicates = saleslt_address[saleslt_address["rowguid"].duplicated(keep=False)]
len(duplicates)

0

In [69]:
# making sure all uuids are valid
import uuid
def is_valid(val):
    try:
        uuid.UUID(str(val))
        return True
    except ValueError:
        return False
        
saleslt_address['isvalid']=saleslt_address['rowguid'].apply(is_valid)

In [70]:
saleslt_address['isvalid'].value_counts()

isvalid
True    450
Name: count, dtype: int64

In [71]:
saleslt_address.drop(columns='isvalid')

,AddressID,AddressLine1,AddressLine2,City,StateProvince,CountryRegion,PostalCode,rowguid,ModifiedDate
0,9,8713 Yosemite Ct.,Unknown,Bothell,Washington,USA,98011,268AF621-76D7-4C78-9441-144FD139821A,2006-07-01
1,11,1318 Lasalle Street,Unknown,Bothell,Washington,USA,98011,981B3303-ACA2-49C7-9A96-FB670785B269,2007-04-01
2,25,9178 Jumping St.,Unknown,Dallas,Texas,USA,75201,C8DF3BD9-48F0-4654-A8DD-14A67A84D3C6,2006-09-01
3,28,9228 Via Del Sol,Unknown,Phoenix,Arizona,USA,85004,12AE5EE1-FC3E-468B-9B92-3B970B169774,2005-09-01
4,32,26910 Indela Road,Unknown,Montreal,Quebec,CAN,H1Y 2H5,84A95F62-3AE8-4E7E-BBD5-5A6F00CD982D,2006-08-01
...,...,...,...,...,...,...,...,...,...
445,1104,9927 N. Main St.,Unknown,Tooele,Utah,USA,84074,CE0FA600-8761-402C-A6C0-84FB6244984B,2005-08-01
446,1105,566 S. Main,Unknown,Cedar City,Utah,USA,84720,C16F92AB-817F-42D1-A560-BC0F86FE2783,2006-07-01
447,11380,165 North Main,Unknown,Austin,Texas,USA,78701,412E3088-1FB8-4A1F-B3A8-ABB23A4F01C1,2006-07-01
448,11381,2000 300th Street,Unknown,Denver,Colorado,USA,80203,A9DBB460-A624-4E6A-AA22-311EE18F6746,2005-09-01


In [72]:
saleslt_address['full_address'] = (
      saleslt_address['AddressLine1'] + ' ' + 
      saleslt_address['AddressLine2']+ ', ' +
      saleslt_address['City'] + ', ' + 
      saleslt_address['StateProvince'] + ', ' + 
      saleslt_address['CountryRegion'] + ' ' + 
      saleslt_address['PostalCode']
  )

saleslt_address


,AddressID,AddressLine1,AddressLine2,City,StateProvince,CountryRegion,PostalCode,rowguid,ModifiedDate,isvalid,full_address
0,9,8713 Yosemite Ct.,Unknown,Bothell,Washington,USA,98011,268AF621-76D7-4C78-9441-144FD139821A,2006-07-01,True,"8713 Yosemite Ct. Unknown, Bothell, Washington..."
1,11,1318 Lasalle Street,Unknown,Bothell,Washington,USA,98011,981B3303-ACA2-49C7-9A96-FB670785B269,2007-04-01,True,"1318 Lasalle Street Unknown, Bothell, Washingt..."
2,25,9178 Jumping St.,Unknown,Dallas,Texas,USA,75201,C8DF3BD9-48F0-4654-A8DD-14A67A84D3C6,2006-09-01,True,"9178 Jumping St. Unknown, Dallas, Texas, USA 7..."
3,28,9228 Via Del Sol,Unknown,Phoenix,Arizona,USA,85004,12AE5EE1-FC3E-468B-9B92-3B970B169774,2005-09-01,True,"9228 Via Del Sol Unknown, Phoenix, Arizona, US..."
4,32,26910 Indela Road,Unknown,Montreal,Quebec,CAN,H1Y 2H5,84A95F62-3AE8-4E7E-BBD5-5A6F00CD982D,2006-08-01,True,"26910 Indela Road Unknown, Montreal, Quebec, C..."
...,...,...,...,...,...,...,...,...,...,...,...
445,1104,9927 N. Main St.,Unknown,Tooele,Utah,USA,84074,CE0FA600-8761-402C-A6C0-84FB6244984B,2005-08-01,True,"9927 N. Main St. Unknown, Tooele, Utah, USA 84074"
446,1105,566 S. Main,Unknown,Cedar City,Utah,USA,84720,C16F92AB-817F-42D1-A560-BC0F86FE2783,2006-07-01,True,"566 S. Main Unknown, Cedar City, Utah, USA 84720"
447,11380,165 North Main,Unknown,Austin,Texas,USA,78701,412E3088-1FB8-4A1F-B3A8-ABB23A4F01C1,2006-07-01,True,"165 North Main Unknown, Austin, Texas, USA 78701"
448,11381,2000 300th Street,Unknown,Denver,Colorado,USA,80203,A9DBB460-A624-4E6A-AA22-311EE18F6746,2005-09-01,True,"2000 300th Street Unknown, Denver, Colorado, U..."


Moving on to the next table

In [73]:
saleslt_customer=pd.read_sql('select* from SalesLT.Customer',conn)

C:\Users\HP\AppData\Local\Temp\ipykernel_21192\2946644435.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  saleslt_customer=pd.read_sql('select* from SalesLT.Customer',conn)


In [74]:
saleslt_customer.head()

,CustomerID,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,CompanyName,SalesPerson,EmailAddress,Phone,PasswordHash,PasswordSalt,rowguid,ModifiedDate
0,1,False,Mr.,Orlando,N.,Gee,None,A Bike Store,adventure-works\pamela0,orlando0@adventure-works.com,245-555-0173,L/Rlwxzp4w7RWmEgXX+/A7cXaePEPcp+KwQhl2fJL7w=,1KjXYs4=,3F5AE95E-B87D-4AED-95B4-C3797AFCB74F,2005-08-01
1,2,False,Mr.,Keith,None,Harris,None,Progressive Sports,adventure-works\david8,keith0@adventure-works.com,170-555-0127,YPdtRdvqeAhj6wyxEsFdshBDNXxkCXn+CRgbvJItknw=,fs1ZGhY=,E552F657-A9AF-4A7D-A645-C429D6E02491,2006-08-01
2,3,False,Ms.,Donna,F.,Carreras,None,Advanced Bike Components,adventure-works\jillian0,donna0@adventure-works.com,279-555-0130,LNoK27abGQo48gGue3EBV/UrlYSToV0/s87dCRV7uJk=,YTNH5Rw=,130774B1-DB21-4EF3-98C8-C104BCD6ED6D,2005-09-01
3,4,False,Ms.,Janet,M.,Gates,None,Modular Cycle Systems,adventure-works\jillian0,janet1@adventure-works.com,710-555-0173,ElzTpSNbUW1Ut+L5cWlfR7MF6nBZia8WpmGaQPjLOJA=,nm7D5e4=,FF862851-1DAA-4044-BE7C-3E85583C054D,2006-07-01
4,5,False,Mr.,Lucy,None,Harrington,None,Metropolitan Sports Supply,adventure-works\shu0,lucy0@adventure-works.com,828-555-0186,KJqV15wsX3PG8TS5GSddp6LFFVdd3CoRftZM/tP0+R4=,cNFKU4w=,83905BDC-6F5E-4F71-B162-C98DA069F38A,2006-09-01


In [75]:
saleslt_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 847 entries, 0 to 846
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   CustomerID    847 non-null    int64         
 1   NameStyle     847 non-null    bool          
 2   Title         840 non-null    object        
 3   FirstName     847 non-null    object        
 4   MiddleName    504 non-null    object        
 5   LastName      847 non-null    object        
 6   Suffix        48 non-null     object        
 7   CompanyName   847 non-null    object        
 8   SalesPerson   847 non-null    object        
 9   EmailAddress  847 non-null    object        
 10  Phone         847 non-null    object        
 11  PasswordHash  847 non-null    object        
 12  PasswordSalt  847 non-null    object        
 13  rowguid       847 non-null    object        
 14  ModifiedDate  847 non-null    datetime64[ns]
dtypes: bool(1), datetime64[ns](1), int64(1),

In [76]:
duplicates=saleslt_customer[saleslt_customer.duplicated(subset=['CustomerID'],keep=False)]

print(len(duplicates)) 
# making sure there are no duplicates in the primary key

0


In [77]:
#replacing null values
saleslt_customer['Title'].fillna('Unknown',inplace=True)
saleslt_customer['MiddleName'].fillna('Unknown',inplace=True)
saleslt_customer['Suffix'].fillna('Unknown',inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_21192\1841748775.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  saleslt_customer['Title'].fillna('Unknown',inplace=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_21192\1841748775.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

In [78]:
#making sure all in standeralized casing and no unecessary white spaces
saleslt_customer["Title"] = saleslt_customer["Title"].str.strip().str.capitalize()
saleslt_customer["FirstName"] = saleslt_customer["FirstName"].str.strip().str.capitalize()
saleslt_customer["MiddleName"] = saleslt_customer["MiddleName"].str.strip().str.capitalize()
saleslt_customer["LastName"] = saleslt_customer["LastName"].str.strip().str.capitalize()
saleslt_customer["CompanyName"] = saleslt_customer["CompanyName"].str.strip().str.capitalize()

In [79]:
saleslt_customer[saleslt_customer['Suffix']!='Unknown']

,CustomerID,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,CompanyName,SalesPerson,EmailAddress,Phone,PasswordHash,PasswordSalt,rowguid,ModifiedDate
9,12,False,Mr.,Johnny,A.,Caprio,Jr.,Bikes and motorbikes,adventure-works\garrett1,johnny0@adventure-works.com,112-555-0191,jtF9jBoFYeJTaET7x+eJDkd7BzMz15Wo9odbGPBaIak=,wVLnvHo=,947BCAF1-1F32-44F3-B9C3-0011F95FBE54,2006-08-01
10,16,False,Mr.,Christopher,R.,Beck,Jr.,Bulk discount store,adventure-works\jae0,christopher1@adventure-works.com,1 (11) 500 555-0132,sKt9daCzEEKWAzivEGPOp8tmaM1R3I+aJfcBjzJRFLo=,8KfYx/4=,C9381589-D31C-4EFE-8978-8D3449EB1F0F,2006-09-01
95,151,False,Mr.,Walter,J.,Brian,IV,Futuristic bikes,adventure-works\shu0,walter0@adventure-works.com,163-555-0155,NN11OHHxrOm6r8lPyhXuBk13QMD+8tph5PIdJ4t0JmY=,PiILPfc=,7F2891F7-DD88-470B-95AE-CC9A7D732B1F,2006-09-01
98,156,False,Mr.,Lester,J.,Bowman,Sr.,Metal processing company,adventure-works\garrett1,lester0@adventure-works.com,844-555-0112,+CFHIbYlE+nsmSbCpJOTLZXChbtYA9JzdSnd1Qejk5g=,w7ec7iI=,CFFFAAFB-98DC-4392-84F9-FB3229F35265,2005-09-01
108,173,False,Mr.,Rudolph,J.,Dillon,Sr.,Sundry sporting goods,adventure-works\josé1,rudolph0@adventure-works.com,722-555-0169,hynkaEM/LxC86JGG4q4AGgiar6iWoIu5bLVB3gFq2RM=,DUDd/5g=,FDC12E98-898B-4C22-972F-F0EBFBFCB7B0,2005-09-01
113,182,False,Mr.,Stanley,A.,Alan,Jr.,Another bicycle company,adventure-works\david8,stanley0@adventure-works.com,156-555-0126,uzR3iWUJwdMRenhBxExmLfmCRPenWgwazHnSEPDbzb8=,kj/7VTw=,D3DA2FF4-51C0-4FCB-A936-1B910458D752,2007-09-01
155,245,False,Mr.,James,L.,Mccoy,II,Leather seat factory,adventure-works\josé1,james12@adventure-works.com,191-555-0151,7fCBS9M7J3qJvmN3sgO/fbT2ap3vnYQ/k+fKjaHYP/0=,r9owTtw=,4A122BDD-4393-42BC-AD49-37E43A0414F6,2006-07-01
166,263,False,Mr.,Timothy,B.,Burnett,Jr.,Farthest bike store,adventure-works\josé1,timothy0@adventure-works.com,251-555-0172,lYPMOabTmO7waFR155ZAm1jDwBg6VW2xrT6sZ1yxPio=,e8KtQ70=,F665A873-61A2-4B21-B9FF-B5903BEAF759,2007-08-01
188,295,False,Mr.,Joseph,P.,Mitzner,Jr.,Engineered bike systems,adventure-works\shu0,joseph4@adventure-works.com,129-555-0164,7obarvC3BHLIE/oFo0n+bkB/EZ03Zc8Vu+F2jpK/CNM=,k6zgROU=,BB5324B1-9ACE-4AAE-8711-41090EDCE4E0,2007-09-01
199,312,False,Mr.,Thomas,B.,Armstrong,Jr.,Resale services,adventure-works\linda3,thomas1@adventure-works.com,226-555-0146,8ml/xroz0hroeUDl/KW2y2QlIIgHgygotLw0Md60cBY=,hXYQBis=,067BB7DA-557D-4145-910C-EE0C82CAA0E4,2005-08-01


In [80]:
#makinf sure all emails are valid
def is_valid_email(email):
    try:
        return '@' in email and '.'in email.split('@')[1]
    except:
        return False

In [81]:
saleslt_customer['EmailAddress'].apply(is_valid_email)

0      True
1      True
2      True
3      True
4      True
       ... 
842    True
843    True
844    True
845    True
846    True
Name: EmailAddress, Length: 847, dtype: bool

In [82]:
duplicate=saleslt_customer[saleslt_customer.duplicated(subset=['PasswordHash'],keep=False)]

In [83]:
len(duplicate)

814

In [84]:
duplicate=saleslt_customer[saleslt_customer.duplicated(subset=['PasswordSalt'],keep=False)]
len(duplicate)

814

In [85]:
#PasswordHash and PasswordSalt are synthetic demo values and may contain duplicates.
#They are not cleaned further since they are irrelevant for business analysis

In [86]:
saleslt_customer['rowguid'].apply(is_valid)

0      True
1      True
2      True
3      True
4      True
       ... 
842    True
843    True
844    True
845    True
846    True
Name: rowguid, Length: 847, dtype: bool

In [87]:
import re

saleslt_customer["CleanPhone"] = saleslt_customer["Phone"].apply(
    lambda x: re.sub(r'\D', '', str(x)) if pd.notna(x) else None
)

saleslt_customer["ValidPhone"] = saleslt_customer["CleanPhone"].apply(
    lambda x: len(x) >= 10 if x else False
)


In [88]:
saleslt_customer['ValidPhone'].value_counts()

ValidPhone
True    847
Name: count, dtype: int64

In [89]:
saleslt_customer.drop(columns=['ValidPhone'],inplace=True)

In [90]:
saleslt_customer['FullName']=(saleslt_customer['FirstName']+' '+saleslt_customer['MiddleName']+' '+saleslt_customer['LastName'])

In [91]:
saleslt_customer

,CustomerID,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,CompanyName,SalesPerson,EmailAddress,Phone,PasswordHash,PasswordSalt,rowguid,ModifiedDate,CleanPhone,FullName
0,1,False,Mr.,Orlando,N.,Gee,Unknown,A bike store,adventure-works\pamela0,orlando0@adventure-works.com,245-555-0173,L/Rlwxzp4w7RWmEgXX+/A7cXaePEPcp+KwQhl2fJL7w=,1KjXYs4=,3F5AE95E-B87D-4AED-95B4-C3797AFCB74F,2005-08-01,2455550173,Orlando N. Gee
1,2,False,Mr.,Keith,Unknown,Harris,Unknown,Progressive sports,adventure-works\david8,keith0@adventure-works.com,170-555-0127,YPdtRdvqeAhj6wyxEsFdshBDNXxkCXn+CRgbvJItknw=,fs1ZGhY=,E552F657-A9AF-4A7D-A645-C429D6E02491,2006-08-01,1705550127,Keith Unknown Harris
2,3,False,Ms.,Donna,F.,Carreras,Unknown,Advanced bike components,adventure-works\jillian0,donna0@adventure-works.com,279-555-0130,LNoK27abGQo48gGue3EBV/UrlYSToV0/s87dCRV7uJk=,YTNH5Rw=,130774B1-DB21-4EF3-98C8-C104BCD6ED6D,2005-09-01,2795550130,Donna F. Carreras
3,4,False,Ms.,Janet,M.,Gates,Unknown,Modular cycle systems,adventure-works\jillian0,janet1@adventure-works.com,710-555-0173,ElzTpSNbUW1Ut+L5cWlfR7MF6nBZia8WpmGaQPjLOJA=,nm7D5e4=,FF862851-1DAA-4044-BE7C-3E85583C054D,2006-07-01,7105550173,Janet M. Gates
4,5,False,Mr.,Lucy,Unknown,Harrington,Unknown,Metropolitan sports supply,adventure-works\shu0,lucy0@adventure-works.com,828-555-0186,KJqV15wsX3PG8TS5GSddp6LFFVdd3CoRftZM/tP0+R4=,cNFKU4w=,83905BDC-6F5E-4F71-B162-C98DA069F38A,2006-09-01,8285550186,Lucy Unknown Harrington
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842,30113,False,Mr.,Raja,D.,Venugopal,Unknown,Nearby cycle shop,adventure-works\jae0,raja0@adventure-works.com,1 (11) 500 555-0195,lx5a4+AFGzH6mzjz6hpiR9scxbYGX8OcqFpv/cI6yFs=,hSwmBWk=,84975C44-DC82-49D3-BFC4-8F31B4C8E994,2006-09-01,1115005550195,Raja D. Venugopal
843,30115,False,Ms.,Dora,P.,Verdad,Unknown,Retreat inn,adventure-works\josé1,dora0@adventure-works.com,155-555-0140,RgzxIE8hSY/z6iJnmDYMZWdJTQBRaWKI7UaZhAPhPxY=,jYD20wc=,E4CF8FD5-30A4-4B8E-8FD8-47032E255778,2006-08-01,1555550140,Dora P. Verdad
844,30116,False,Ms.,Wanda,F.,Vernon,Unknown,Technical parts manufacturing,adventure-works\shu0,wanda0@adventure-works.com,433-555-0168,hNQpZV8787KyeDWfllnZWzY+l8DFEYTnwoZZzYtusi0=,HDCU1Bk=,EC409609-D25D-41B8-9D15-A1AA6E89FC77,2007-07-01,4335550168,Wanda F. Vernon
845,30117,False,Mr.,Robert,R.,Vessa,Unknown,Totes & baskets company,adventure-works\jillian0,robert13@adventure-works.com,560-555-0171,UWGC2U8F7AUNA2FuiT4agrBoxAFskHGQSxqP39B7zLQ=,iES3IZA=,6F08E2FB-1CD3-4F6E-A2E6-385669598B19,2005-08-01,5605550171,Robert R. Vessa


In [92]:
saleslt_customeraddress=pd.read_sql('select* from SalesLT.CustomerAddress',conn)


C:\Users\HP\AppData\Local\Temp\ipykernel_21192\1334409223.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  saleslt_customeraddress=pd.read_sql('select* from SalesLT.CustomerAddress',conn)


In [93]:
saleslt_customeraddress.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 417 entries, 0 to 416
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   CustomerID    417 non-null    int64         
 1   AddressID     417 non-null    int64         
 2   AddressType   417 non-null    object        
 3   rowguid       417 non-null    object        
 4   ModifiedDate  417 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 16.4+ KB


In [94]:
# sense it is a weak entity we make sure that these columns together form a primarykey
duplicate= saleslt_customeraddress[saleslt_customeraddress.duplicated(subset=['CustomerID','AddressID'],keep=False)]
len(duplicate)

0

In [95]:
saleslt_customeraddress['rowguid'].apply(is_valid).value_counts()

rowguid
True    417
Name: count, dtype: int64

In [96]:
saleslt_customeraddress['AddressType']=saleslt_customeraddress['AddressType'].str.capitalize()

In [97]:
saleslt_product=pd.read_sql('select* from SalesLT.Product',conn)

C:\Users\HP\AppData\Local\Temp\ipykernel_21192\1066602104.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  saleslt_product=pd.read_sql('select* from SalesLT.Product',conn)


In [98]:
saleslt_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295 entries, 0 to 294
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ProductID               295 non-null    int64         
 1   Name                    295 non-null    object        
 2   ProductNumber           295 non-null    object        
 3   Color                   245 non-null    object        
 4   StandardCost            295 non-null    float64       
 5   ListPrice               295 non-null    float64       
 6   Size                    211 non-null    object        
 7   Weight                  198 non-null    float64       
 8   ProductCategoryID       295 non-null    int64         
 9   ProductModelID          295 non-null    int64         
 10  SellStartDate           295 non-null    datetime64[ns]
 11  SellEndDate             98 non-null     datetime64[ns]
 12  DiscontinuedDate        0 non-null      object    

In [99]:
saleslt_product.drop(columns='DiscontinuedDate',inplace=True)

In [100]:
saleslt_product['Color'].fillna('Unknown',inplace=True)
saleslt_product['Size'].fillna('Unknown',inplace=True)
saleslt_product['Weight'].fillna('Unknown',inplace=True)
saleslt_product['SellEndDate'].fillna('Unknown',inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_21192\389326672.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  saleslt_product['Color'].fillna('Unknown',inplace=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_21192\389326672.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exam

In [101]:
saleslt_product.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295 entries, 0 to 294
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ProductID               295 non-null    int64         
 1   Name                    295 non-null    object        
 2   ProductNumber           295 non-null    object        
 3   Color                   295 non-null    object        
 4   StandardCost            295 non-null    float64       
 5   ListPrice               295 non-null    float64       
 6   Size                    295 non-null    object        
 7   Weight                  295 non-null    object        
 8   ProductCategoryID       295 non-null    int64         
 9   ProductModelID          295 non-null    int64         
 10  SellStartDate           295 non-null    datetime64[ns]
 11  SellEndDate             295 non-null    object        
 12  ThumbNailPhoto          295 non-null    object    

In [102]:
saleslt_product['Name']=saleslt_product['Name'].astype(str).str.capitalize().str.strip()

In [103]:
#all productids are unique
duplicate= saleslt_product[saleslt_product.duplicated(subset=['ProductID'],keep=False)]
len(duplicate)

0

In [104]:
saleslt_product['Color'] = saleslt_product['Color'].str.strip().str.capitalize()


In [105]:
#making sure no negatives in price
negatives=saleslt_product[saleslt_product['StandardCost']<0]
len(negatives)

0

In [106]:
negatives=saleslt_product[saleslt_product['ListPrice']<0]
len(negatives)

0

In [107]:
# no price is less than the standard
notright=saleslt_product[saleslt_product['ListPrice']<saleslt_product['StandardCost']]
len(notright)

0

In [108]:
saleslt_productcategory=pd.read_sql('select * from SalesLT.ProductCategory',conn)
saleslt_productmodel=pd.read_sql('select * from SalesLT.ProductModel',conn)

C:\Users\HP\AppData\Local\Temp\ipykernel_21192\678400435.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  saleslt_productcategory=pd.read_sql('select * from SalesLT.ProductCategory',conn)
C:\Users\HP\AppData\Local\Temp\ipykernel_21192\678400435.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  saleslt_productmodel=pd.read_sql('select * from SalesLT.ProductModel',conn)


In [109]:
saleslt_product['ProductCategoryID'].isin(saleslt_productcategory['ProductCategoryID']).value_counts()

ProductCategoryID
True    295
Name: count, dtype: int64

In [110]:
saleslt_product['ProductModelID'].isin(saleslt_productmodel['ProductModelID']).value_counts()

ProductModelID
True    295
Name: count, dtype: int64

In [111]:
filter = saleslt_product['SellEndDate'] != 'Unknown'
(saleslt_product[filter]['SellStartDate'] < saleslt_product[filter]['SellEndDate']).sum()


np.int64(98)

In [112]:
saleslt_product[saleslt_product['SellEndDate']=='Unknown'].value_counts()
#197+98=295

ProductID  Name                       ProductNumber  Color    StandardCost  ListPrice  Size     Weight   ProductCategoryID  ProductModelID  SellStartDate  SellEndDate  ThumbNailPhoto                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [113]:
saleslt_product['ThumbnailPhotoFileName'] = saleslt_product['ThumbnailPhotoFileName'].str.lower()


In [114]:
mask_invalid=~saleslt_product['ThumbnailPhotoFileName'].str.endswith('.gif')

In [115]:
mask_invalid.value_counts()

ThumbnailPhotoFileName
False    295
Name: count, dtype: int64

In [116]:
saleslt_product['rowguid'].apply(is_valid).value_counts()

rowguid
True    295
Name: count, dtype: int64

In [117]:
saleslt_product['MarginProfit']=saleslt_product['ListPrice']-saleslt_product['StandardCost']

In [118]:
saleslt_product

,ProductID,Name,ProductNumber,Color,StandardCost,ListPrice,Size,Weight,ProductCategoryID,ProductModelID,SellStartDate,SellEndDate,ThumbNailPhoto,ThumbnailPhotoFileName,rowguid,ModifiedDate,MarginProfit
0,680,"Hl road frame - black, 58",FR-R92B-58,Black,1059.3100,1431.50,58,1016.04,18,6,2002-06-01,Unknown,b'GIF89aP\x001\x00\xf7\x00\x00\x00\x00\x00\x80...,no_image_available_small.gif,43DD68D6-14A4-461F-9069-55309D90EA7E,2008-03-11 10:01:36.827,372.1900
1,706,"Hl road frame - red, 58",FR-R92R-58,Red,1059.3100,1431.50,58,1016.04,18,6,2002-06-01,Unknown,b'GIF89aP\x001\x00\xf7\x00\x00\x00\x00\x00\x80...,no_image_available_small.gif,9540FF17-2712-4C90-A3D1-8CE5568B2462,2008-03-11 10:01:36.827,372.1900
2,707,"Sport-100 helmet, red",HL-U509-R,Red,13.0863,34.99,Unknown,Unknown,35,33,2005-07-01,Unknown,b'GIF89aP\x001\x00\xf7\x00\x00\x00\x00\x00\x80...,no_image_available_small.gif,2E1EF41A-C08A-4FF6-8ADA-BDE58B64A712,2008-03-11 10:01:36.827,21.9037
3,708,"Sport-100 helmet, black",HL-U509,Black,13.0863,34.99,Unknown,Unknown,35,33,2005-07-01,Unknown,b'GIF89aP\x001\x00\xf7\x00\x00\x00\x00\x00\x80...,no_image_available_small.gif,A25A44FB-C2DE-4268-958F-110B8D7621E2,2008-03-11 10:01:36.827,21.9037
4,709,"Mountain bike socks, m",SO-B909-M,White,3.3963,9.50,M,Unknown,27,18,2005-07-01,2006-06-30 00:00:00,b'GIF89aP\x001\x00\xf7\x00\x00\x00\x00\x00\x80...,no_image_available_small.gif,18F95F47-1540-4E02-8F1F-CC1BCB6828D0,2008-03-11 10:01:36.827,6.1037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,995,Ml bottom bracket,BB-8107,Unknown,44.9506,101.24,Unknown,168.0,9,96,2007-07-01,Unknown,b'GIF89aP\x001\x00\xf7\x00\x00\x00\x00\x00\x80...,no_image_available_small.gif,71AB847F-D091-42D6-B735-7B0C2D82FC84,2008-03-11 10:01:36.827,56.2894
291,996,Hl bottom bracket,BB-9108,Unknown,53.9416,121.49,Unknown,170.0,9,97,2007-07-01,Unknown,b'GIF89aP\x001\x00\xf7\x00\x00\x00\x00\x00\x80...,no_image_available_small.gif,230C47C5-08B2-4CE3-B706-69C0BDD62965,2008-03-11 10:01:36.827,67.5484
292,997,"Road-750 black, 44",BK-R19B-44,Black,343.6496,539.99,44,8967.47,6,31,2007-07-01,Unknown,b'GIF89aP\x001\x00\xf7\x00\x00\xcd\xc9\xca\xc3...,roadster_black_small.gif,44CE4802-409F-43AB-9B27-CA53421805BE,2008-03-11 10:01:36.827,196.3404
293,998,"Road-750 black, 48",BK-R19B-48,Black,343.6496,539.99,48,9130.77,6,31,2007-07-01,Unknown,b'GIF89aP\x001\x00\xf7\x00\x00\xcd\xc9\xca\xc3...,roadster_black_small.gif,3DE9A212-1D49-40B6-B10A-F564D981DBDE,2008-03-11 10:01:36.827,196.3404


In [119]:
saleslt_productcategory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   ProductCategoryID        41 non-null     int64         
 1   ParentProductCategoryID  37 non-null     float64       
 2   Name                     41 non-null     object        
 3   rowguid                  41 non-null     object        
 4   ModifiedDate             41 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 1.7+ KB


In [120]:
duplicate=saleslt_productcategory[saleslt_productcategory.duplicated(subset=['ProductCategoryID'],keep=False)]
len(duplicate)

0

In [121]:
saleslt_productcategory['ParentProductCategoryID'].fillna('Unknown',inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_21192\1098540632.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  saleslt_productcategory['ParentProductCategoryID'].fillna('Unknown',inplace=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_21192\1098540632.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  saleslt_productcategory['ParentProductCategoryI

In [122]:
saleslt_productcategory['Name']=saleslt_productcategory['Name'].str.capitalize()

In [123]:
saleslt_productcategory['rowguid'].apply(is_valid).value_counts()

rowguid
True    41
Name: count, dtype: int64

In [124]:
saleslt_productmodel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ProductModelID      128 non-null    int64         
 1   Name                128 non-null    object        
 2   CatalogDescription  6 non-null      object        
 3   rowguid             128 non-null    object        
 4   ModifiedDate        128 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 5.1+ KB


In [125]:
saleslt_productmodel['CatalogDescription'].fillna('Unknown',inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_21192\1847726781.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  saleslt_productmodel['CatalogDescription'].fillna('Unknown',inplace=True)


In [126]:
duplicate=saleslt_productmodel[saleslt_productmodel.duplicated(subset=['ProductModelID'],keep=False)]
len(duplicate)

0

In [127]:
saleslt_productmodel['rowguid'].apply(is_valid).value_counts()

rowguid
True    128
Name: count, dtype: int64

In [128]:
saleslt_productmodel['Name']=saleslt_productmodel['Name'].str.capitalize()

In [129]:
saleslt_productmodelproductproductdescription=pd.read_sql('select* from SalesLT.ProductModelProductDescription',conn)

C:\Users\HP\AppData\Local\Temp\ipykernel_21192\823420713.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  saleslt_productmodelproductproductdescription=pd.read_sql('select* from SalesLT.ProductModelProductDescription',conn)


In [130]:
saleslt_productmodelproductproductdescription.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 762 entries, 0 to 761
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ProductModelID        762 non-null    int64         
 1   ProductDescriptionID  762 non-null    int64         
 2   Culture               762 non-null    object        
 3   rowguid               762 non-null    object        
 4   ModifiedDate          762 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 29.9+ KB


In [131]:
duplicate=saleslt_productmodelproductproductdescription[saleslt_productmodelproductproductdescription.duplicated(subset=['ProductModelID','ProductDescriptionID'],keep=False)]
len(duplicate)

0

In [132]:
saleslt_productmodelproductproductdescription['rowguid'].apply(is_valid).value_counts()

rowguid
True    762
Name: count, dtype: int64

In [133]:
duplicate=saleslt_productmodelproductproductdescription[saleslt_productmodelproductproductdescription.duplicated(subset=['ProductModelID','ProductDescriptionID','Culture'],keep=False)]
len(duplicate)

0

In [134]:
saleslt_salesorderdetail=pd.read_sql('select* from SalesLT.SalesOrderDetail',conn)

C:\Users\HP\AppData\Local\Temp\ipykernel_21192\782105400.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  saleslt_salesorderdetail=pd.read_sql('select* from SalesLT.SalesOrderDetail',conn)


In [135]:
saleslt_salesorderdetail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542 entries, 0 to 541
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   SalesOrderID        542 non-null    int64         
 1   SalesOrderDetailID  542 non-null    int64         
 2   OrderQty            542 non-null    int64         
 3   ProductID           542 non-null    int64         
 4   UnitPrice           542 non-null    float64       
 5   UnitPriceDiscount   542 non-null    float64       
 6   LineTotal           542 non-null    float64       
 7   rowguid             542 non-null    object        
 8   ModifiedDate        542 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(4), object(1)
memory usage: 38.2+ KB


In [136]:
duplicate=saleslt_salesorderdetail[saleslt_salesorderdetail.duplicated(subset=['SalesOrderID','SalesOrderDetailID'],keep=False)]
len(duplicate)

0

In [137]:
saleslt_salesorderdetail['rowguid'].apply(is_valid).value_counts()

rowguid
True    542
Name: count, dtype: int64

In [138]:
(saleslt_salesorderdetail['LineTotal']<0).sum()

np.int64(0)

In [139]:
(saleslt_salesorderdetail['UnitPrice']<0).sum()

np.int64(0)

In [140]:
saleslt_salesorderheader=pd.read_sql('select* from SalesLT.SalesOrderHeader',conn)

C:\Users\HP\AppData\Local\Temp\ipykernel_21192\471417913.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  saleslt_salesorderheader=pd.read_sql('select* from SalesLT.SalesOrderHeader',conn)


In [141]:
saleslt_salesorderheader.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   SalesOrderID            32 non-null     int64         
 1   RevisionNumber          32 non-null     int64         
 2   OrderDate               32 non-null     datetime64[ns]
 3   DueDate                 32 non-null     datetime64[ns]
 4   ShipDate                32 non-null     datetime64[ns]
 5   Status                  32 non-null     int64         
 6   OnlineOrderFlag         32 non-null     bool          
 7   SalesOrderNumber        32 non-null     object        
 8   PurchaseOrderNumber     32 non-null     object        
 9   AccountNumber           32 non-null     object        
 10  CustomerID              32 non-null     int64         
 11  ShipToAddressID         32 non-null     int64         
 12  BillToAddressID         32 non-null     int64       

In [142]:
saleslt_salesorderheader.head()


,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,...,BillToAddressID,ShipMethod,CreditCardApprovalCode,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,71774,2,2008-06-01,2008-06-13,2008-06-08,5,False,SO71774,PO348186287,10-4020-000609,...,1092,CARGO TRANSPORT 5,None,880.3484,70.4279,22.0087,972.7850,None,89E42CDC-8506-48A2-B89B-EB3E64E3554E,2008-06-08
1,71776,2,2008-06-01,2008-06-13,2008-06-08,5,False,SO71776,PO19952192051,10-4020-000106,...,640,CARGO TRANSPORT 5,None,78.8100,6.3048,1.9703,87.0851,None,8A3448C5-E677-4158-A29B-DD33069BE0B0,2008-06-08
2,71780,2,2008-06-01,2008-06-13,2008-06-08,5,False,SO71780,PO19604173239,10-4020-000340,...,653,CARGO TRANSPORT 5,None,38418.6895,3073.4952,960.4672,42452.6519,None,A47665D2-7AC9-4CF3-8A8B-2A3883554284,2008-06-08
3,71782,2,2008-06-01,2008-06-13,2008-06-08,5,False,SO71782,PO19372114749,10-4020-000582,...,1086,CARGO TRANSPORT 5,None,39785.3304,3182.8264,994.6333,43962.7901,None,F1BE45A5-5C57-4A50-93C6-5F8BE44CB7CB,2008-06-08
4,71783,2,2008-06-01,2008-06-13,2008-06-08,5,False,SO71783,PO19343113609,10-4020-000024,...,992,CARGO TRANSPORT 5,None,83858.4261,6708.6741,2096.4607,92663.5609,None,7DB2329E-6446-42A8-8915-9C8370B68ED8,2008-06-08


In [143]:
saleslt_salesorderheader=saleslt_salesorderheader.drop(columns=['Comment','CreditCardApprovalCode'])

In [144]:
saleslt_salesorderheader['rowguid'].apply(is_valid).value_counts()

rowguid
True    32
Name: count, dtype: int64

In [145]:
duplicate=saleslt_salesorderheader[saleslt_salesorderheader.duplicated(subset=['SalesOrderID'],keep=False)]
len(duplicate)

0

In [146]:
saleslt_salesorderheader['isvalid']=(
saleslt_salesorderheader['TotalDue']==(saleslt_salesorderheader['TaxAmt']
                                       +saleslt_salesorderheader['Freight']
                                       +saleslt_salesorderheader['SubTotal'])
)

In [147]:
saleslt_salesorderheader.loc[saleslt_salesorderheader['isvalid'] == False, 'TotalDue'] = (
    saleslt_salesorderheader['Freight'] 
    + saleslt_salesorderheader['SubTotal'] 
    + saleslt_salesorderheader['TaxAmt']
)


In [148]:
saleslt_salesorderheader=saleslt_salesorderheader.drop(columns=['isvalid'])

In [149]:
(saleslt_salesorderheader['ShipDate']<saleslt_salesorderheader['DueDate']).sum()

np.int64(32)

In [162]:
saleslt_productdescription=pd.read_sql('select* from SalesLT.ProductDescription',conn)

C:\Users\HP\AppData\Local\Temp\ipykernel_21192\314897811.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  saleslt_productdescription=pd.read_sql('select* from SalesLT.ProductDescription',conn)


In [164]:
saleslt_productdescription.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 762 entries, 0 to 761
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ProductDescriptionID  762 non-null    int64         
 1   Description           762 non-null    object        
 2   rowguid               762 non-null    object        
 3   ModifiedDate          762 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 23.9+ KB


In [163]:
saleslt_productdescription.head()

,ProductDescriptionID,Description,rowguid,ModifiedDate
0,3,Chromoly steel.,301EED3A-1A82-4855-99CB-2AFE8290D641,2007-06-01
1,4,Aluminum alloy cups; large diameter spindle.,DFEBA528-DA11-4650-9D86-CAFDA7294EB0,2007-06-01
2,5,Aluminum alloy cups and a hollow axle.,F7178DA7-1A7E-4997-8470-06737181305E,2007-06-01
3,8,"Suitable for any type of riding, on or off-roa...",8E6746E5-AD97-46E2-BD24-FCEA075C3B52,2007-06-01
4,64,This bike delivers a high-level of performance...,7B1C4E90-85E2-4792-B47B-E0C424E2EC94,2007-06-01


In [166]:
duplicate=saleslt_productdescription[saleslt_productdescription.duplicated(subset=['ProductDescriptionID'],keep=False)]
len(duplicate)

0

In [169]:
saleslt_productdescription['rowguid'].apply(is_valid).value_counts()

rowguid
True    762
Name: count, dtype: int64

In [153]:
import pyodbc
from sqlalchemy import create_engine

engine = create_engine(
    "mssql+pyodbc://DESKTOP-UR57B5H\\SQLEXPRESS/CleanedAdventureWorks"
    "?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"
)



In [155]:
saleslt_address.to_sql(
    "SalesLT_Address",   
    con=engine,
    if_exists="replace", 
    index=False
)



70

In [156]:
saleslt_customer.to_sql(
    "SalesLT_Customer",   
    con=engine,
    if_exists="replace", 
    index=False
)

109

In [157]:
saleslt_customeraddress.to_sql(
    "SalesLT_CustomerAddress",   
    con=engine,
    if_exists="replace", 
    index=False
)

417

In [158]:
saleslt_product.to_sql(
    "SalesLT_Product",   
    con=engine,
    if_exists="replace", 
    index=False
)

49

In [159]:
saleslt_productcategory.to_sql(
    "SalesLT_ProductCategory",   
    con=engine,
    if_exists="replace", 
    index=False
)

41

In [170]:
saleslt_productdescription.to_sql(
    "SalesLT_ProductDescription",   
    con=engine,
    if_exists="replace", 
    index=False
)

238

In [171]:
saleslt_productmodel.to_sql(
    "SalesLT_ProductModel",   
    con=engine,
    if_exists="replace", 
    index=False
)

128

In [172]:
saleslt_productmodelproductproductdescription.to_sql(
    "SalesLT_ProductModelProductDescription",   
    con=engine,
    if_exists="replace", 
    index=False
)

343

In [173]:
saleslt_salesorderdetail.to_sql(
    "SalesLT_SalesOrderDetail",   
    con=engine,
    if_exists="replace", 
    index=False
)

76

In [174]:
saleslt_salesorderheader.to_sql(
    "SalesLT_SalesOrderHeader",   
    con=engine,
    if_exists="replace", 
    index=False
)

32